# Building `makemore` Part 3:
**Activations, Gradients, BatchNorm**

https://www.youtube.com/watch?v=P6sfmUTpUmc&list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ&index=4

https://youtu.be/P6sfmUTpUmc?si=NaDZmPDVSSVEshtF&t=83